In [ ]:

import os
import time
import numpy as np
import cv2 as cv
import mediapipe as mp
from matplotlib import pyplot as plt

# Constants
DELAY = 50
INPUT_TYPES = ['palm', 'fist', 'thumbsup', 'gun', 'call']
MIN_INPUT_COUNT = 2000 + DELAY
INPUT_PATH = 'Inputs/'

# Make directories
def make_dirs():
    if not os.path.isdir(INPUT_PATH):
        os.mkdir(INPUT_PATH, mode=511)
    os.chdir(INPUT_PATH)
    for dirs in INPUT_TYPES:
        if not os.path.isdir(dirs):
            os.mkdir(dirs, mode=511)
    os.chdir('..')

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

# Capture from webcam
webcam = 0
capture = cv.VideoCapture(webcam)
fps = int(capture.get(cv.CAP_PROP_FPS))
print("fps is " + str(fps))
_, frame = capture.read()
height, width, _ = frame.shape

# Loop through input types
for types in INPUT_TYPES:
    count = 0
    while capture.isOpened():
        if (cv.waitKey(1) & 0xFF == 13) or (count >= MIN_INPUT_COUNT + 1):
            break
        black = np.zeros(shape=frame.shape)
        _, frame = capture.read()
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        result = holistic.process(frame_rgb)
capture.release()
cv.destroyAllWindows()
